# Spin up EC2 instances

In [ ]:
import boto3
import time

n_instances = 4

session = boto3.Session()

ec2 = session.resource('ec2')
ec2_client = session.client('ec2')

instances = ec2.create_instances(ImageId='ami-083654bd07b5da81d',
                                 MinCount=1,
                                 MaxCount=n_instances,
                                 InstanceType='m5.large',
                                 KeyName='mac30123',
                                 SecurityGroupIds=['sg-0909ca9f17b315453'],
                                 SecurityGroups=['q3-test-policy'],
                                 IamInstanceProfile=
                                     {'Name': 'EMR_EC2_DefaultRole'},
                                )

# Wait until EC2 instances are running before moving on
waiter = ec2_client.get_waiter('instance_running')
waiter.wait(InstanceIds=[instance.id for instance in instances])

# Get instances list
instance_dns = [instance.public_dns_name 
                 for instance in ec2.instances.all() 
                 if instance.state['Name'] == 'running'
               ]

import paramiko
from scp import SCPClient

ssh_list = []

for instance in range(n_instances):
    ssh = paramiko.SSHClient()
    ssh_list.append(ssh)
    
time.sleep(60)

In [20]:
# Copy over relevant files and then restart ssh because you need to exit and re-enter to config usermod
stdin, stdout, stderr = [[None, None] for i in range(3)]
for instance in range(n_instances):
    ssh = ssh_list[instance]
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(instance_dns[instance],
                username = 'ubuntu',
                key_filename='/Users/bli/mac30123.pem')
    
    with SCPClient(ssh.get_transport()) as scp:
        scp.put("install_docker.sh")
        
    stdin, stdout, stderr = ssh.exec_command("bash install_docker.sh " + str(instance))

    print("docker installation status for instance " + str(instance) + ":")
    print(stdout.channel.recv_exit_status())

    ssh.close()

docker installation status for instance 0:
0
docker installation status for instance 1:
0
docker installation status for instance 2:
0
docker installation status for instance 3:
0


In [22]:
for instance_id in instance_dns:
    print('ssh -i "mac30123.pem" ubuntu@' + instance_id)

ssh -i "mac30123.pem" ubuntu@ec2-54-90-88-251.compute-1.amazonaws.com
ssh -i "mac30123.pem" ubuntu@ec2-34-227-46-141.compute-1.amazonaws.com
ssh -i "mac30123.pem" ubuntu@ec2-3-87-124-205.compute-1.amazonaws.com
ssh -i "mac30123.pem" ubuntu@ec2-3-80-95-15.compute-1.amazonaws.com


In [ ]:
# Terminate EC2 Instances:
ec2_client.terminate_instances(InstanceIds=[instance.id for instance in instances])

# Confirm that EC2 instances were terminated:
waiter = ec2_client.get_waiter('instance_terminated')
waiter.wait(InstanceIds=[instance.id for instance in instances])
print("EC2 Instances Successfully Terminated")